In [154]:
# importing the module
import cv2
import math
from win32api import GetSystemMetrics
import time
import numpy as np
import os

In [155]:
# Image path
imagePath = 'Images'
imageName = "sample3.PNG"
imagePath = os.path.join(imagePath,imageName)
resizedImageName = imageName + "_resized"
#road detection works better with images where the scale is smaller per unit pixel 
#(screen distance = lesser real world distance)
#It also works better with a different style of image

In [156]:
scalePoints = []
mountingPoints = []

buffer_length = 0   #buffer length before the midpoint from which viewing must begin
theta = (66.75*math.pi)/180    #diagonal angle FOV of camera (GIVEN!!)
phi = 2*math.atan(0.8*math.tan(theta/2))  #angle of view larger side of camera resolution (4 in 4:3)
omega = 2*math.atan(0.6*math.tan(theta/2))     #angle of view larger side of camera resolution (3 in 4:3)
alpha = (75*math.pi)/180   #set later on in the code based on the height of the camera [angle of camera from negative z axis]

minArea = 1000

fp1 =0
fp2 =0
fp3 =0
fp4 =0

In [157]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [158]:
def getResizedForDisplayImg(img):
    screen_w, screen_h = GetSystemMetrics(0), GetSystemMetrics(1)
    #print("screen size",screen_w, screen_h)
    h,w,channel_nbr = img.shape
    # img get w of screen and adapt h
    h = h * (screen_w / w)
    w = screen_w
    if h > screen_h: #if img h still too big
        # img get h of screen and adapt w
        w = w * (screen_h / h)
        h = screen_h
    w, h = w*0.9, h*0.9 # because you don't want it to be that big, right ?
    w, h = int(w), int(h) # you need int for the cv2.resize
    return cv2.resize(img, (w, h))

In [159]:
def getRoadsHSV(img):

    temp_img = img.copy()
    hsv = img.copy()
    hsv = cv2.cvtColor(hsv, cv2.COLOR_BGR2HSV)

    lower = np.array([255, 0, 0], dtype="uint8")
    # upper = np.array([121, 255, 255], dtype="uint8")
    mask = cv2.inRange(temp_img, lower, lower)
    final = cv2.bitwise_and(temp_img,temp_img,mask=mask)

    # cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    # for c in cnts:
    #     for eps in np.linspace(0.0001, 0.0002, 5):
    #         # approximate the contour
    #         peri = cv2.arcLength(c, True)
    #         approx = cv2.approxPolyDP(c, eps * peri, True)
    #     cv2.drawContours(temp_img, [approx], -1, (36, 255, 12), 2)

    return final

In [160]:
def getMountingPoints(img):

    imageCopy = img.copy()

    # Upper and lower color limit customized for snazzy maps
    low_red = (55, 55, 255)

    # create masks
    red_mask = cv2.inRange(imageCopy, low_red, low_red)
    
    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_DILATE,kernel)

    # convert to grayscale
    # operatedImage = cv2.cvtColor(combined_mask, cv2.COLOR_BGR2GRAY)

    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) #uncomment if you feel image is noisy (not needed)
    # setting to 32-bit floating point
    operatedImage = np.float32(combined_mask)

    # apply the cv2.cornerHarris method
    # to detect the corners with appropriate values as input parameters
    dest = cv2.cornerHarris(operatedImage, 2, 3, 0.04)

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # draw on the output image
    imageCopy[dest > 0.01 * dest.max()]=[255, 255, 255]

    return imageCopy

In [161]:
# Make the corners into one point
def cluster2Point(clusterImg, img):

    # innitates the the single point corner arrays
    Points = []

    # blank = np.zeros(img.shape, dtype='uint8')
    imageCopy = img.copy()

    # create masks for the corner clusters
    white_mask = cv2.inRange(clusterImg, (255,255,255), (255,255,255))

    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(white_mask, cv2.MORPH_DILATE,kernel)


    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for i in cnts:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
        # print(f"center - {cx},{cy}")
        Points.append([cx,cy])
        imageCopy[cy, cx] = [255, 255, 255]
        # cv2.circle(imageCopy, (cx, cy), 1, (255, 255, 255), -1)
        # cv2.drawContours(blank, [i], -1, 0, -1)

    return Points, imageCopy

In [162]:
def getRoadsnParkings(img):
    
    tempImg = img.copy()
    # define color ranges
    # blue_lower = (250,0,0)
    blue = np.array([255, 0, 0], dtype="uint8")

    # create masks
    blue_mask = cv2.inRange(img, blue, blue)

    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    #combined_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_DILATE,kernel)


    # findcontours
    cnts=cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    mask = np.ones(img.shape[:2], dtype="uint8") * 255
    for c in cnts:
        # area = cv2.contourArea(c)
        # for eps in np.linspace(0.001, 0.005, 5):
        #     # approximate the contour
        #     peri = cv2.arcLength(c, True)
        #     approx = cv2.approxPolyDP(c, eps * peri, True)
        cv2.drawContours(tempImg, [c], -1, 0, -1)

    mask = cv2.bitwise_not(mask)

    #image = cv2.bitwise_and(img, img, mask=mask)
    return blue_mask

In [163]:
def get_MP_FromBuildings_ButAlsoText(img):

    # Upper and lower color limit
    # low_yellow = (239,248,253)
    # high_yellow = (243,252,255)

    # low_gray = (241,241,241)
    # high_gray = (244,243,241)

    low_red = (55, 55, 255)
    high_red = (55, 55, 255)

    # create masks
    # yellow_mask = cv2.inRange(img, low_yellow, high_yellow )
    # gray_mask = cv2.inRange(img, low_gray, high_gray)
    red_mask = cv2.inRange(img, low_red, high_red)

    # combine masks
    # combined_mask = cv2.bitwise_or(yellow_mask, gray_mask)
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_DILATE,kernel)

    # findcontours
    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    output = img.copy()
    for c in cnts:
        # cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)
        area = cv2.contourArea(c)
        if area > 500:
            for eps in np.linspace(0.001, 0.008, 10):
                # approximate the contour
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps * peri, True)
            
            # draw the approximated contour on the image  
            cv2.drawContours(output, [approx], -1, (0, 255, 0), thickness=1)
            #cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)

    # write points in mounting list
    # mountingPoints = []
    # for y in range(len(img)):
    #     for x in range(len(img[y])):
    #         b = img[y, x, 0]
    #         g = img[y, x, 1]
    #         r = img[y, x, 2]
    #         if(b == 255 and g == 0 and r == 255):
    #             mountingPoints.append((x,y))

    # return mountingPoints
    return output

In [164]:
def pointGen(source, m, l):
    # m is the slope of line, and the
    # required Point lies distance l
    # away from the source Point
    a = Point(0, 0)
    b = Point(0, 0)

    # slope is 0
    if m == 0:
        a.x = source.x + l
        a.y = source.y

        b.x = source.x - l
        b.y = source.y

    # if slope is infinite
    elif math.isfinite(m) is False:
        a.x = source.x
        a.y = source.y + l

        b.x = source.x
        b.y = source.y - l
    else:
        dx = (l / math.sqrt(1 + (m * m)))
        dy = m * dx
        a.x = source.x + dx
        a.y = source.y + dy
        b.x = source.x - dx
        b.y = source.y - dy
    
    return [[a.x,a.y],[b.x,b.y]]

In [165]:
def detectCorner(image):

    # making a copy of the image to have the original image untouched in main loop
    imageSub = image.copy()

    # convert to gray and perform Harris corner detection
    gray = cv2.cvtColor(imageSub,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    
    #~~~~~~~~~~~~~#
    #for obtaining mounting points from red buildings img
    #~~~~~~~~~~~~~#
    dst = cv2.cornerHarris(gray,2,3,0.04)

    # result is dilated for marking the corners, not important
    dst = cv2.dilate(dst,None)

    # threshold for an optimal value, it may vary depending on the image.
    imageSub[dst>0.01*dst.max()]=[0,0,255]

    return imageSub

In [166]:
def click_MountingnScale_points(event, x, y, flags, params):

    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        scalePoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +str(y), (x,y), font,1, (255, 255, 0), 2)
        cv2.imshow('image', img)


    # checking for right mouse clicks    
    if event==cv2.EVENT_RBUTTONDOWN:
        scalePoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        b = img[y, x, 0]
        g = img[y, x, 1]
        r = img[y, x, 2]
        cv2.putText(img, str(b) + ',' +str(g) + ',' + str(r),(x,y), font, 1,(255, 255, 0), 2)
        cv2.imshow('image', img)

In [167]:
# reading the image
img = cv2.imread(imagePath)

# resizing and reading the resized the image
# img_temp = getResizedForDisplayImg(img)
# cv2.imwrite(os.path.join(imagePath, resizedImageName+".PNG"), img_temp)
# img = cv2.imread(imagePath)


# displaying the image
cv2.imshow('image', img)

# setting mouse handler for the image
cv2.setMouseCallback('image', click_MountingnScale_points)

# exit image window
cv2.waitKey(0)
cv2.destroyAllWindows()

# Scale for pix to meter conversion
# scaleConst = int(input('Scale: '))
scaleConst = 10

# Get mounting points from building borders
# mountingPoints = get_MP_FromBuildings_ButAlsoText(img)

# Pixel distance of scale in image
# actual distance(m) = (scale constant)*(obtained magnitude)/scale
scale = abs(pow(pow(scalePoints[0][0] - scalePoints[1][0],2) + pow(scalePoints[0][1] - scalePoints[1][1],2),0.5))

# heights = [x*0.1 for x in range(30,60)] #average height of light poles is 9 to 14 feet ~ 4.2m max
heights = [h*0.1 for h in range(30, 60)]
# Converting height in meter array to height in pixel array
heightPix = [h*scale/scaleConst for h in heights]

In [168]:
# reading the resized the image
img = cv2.imread(imagePath)

# obtaining the mask of the roads and parkings in the map image
road = getRoadsnParkings(img)

In [169]:
cv2.imshow('win', road)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [170]:
img = cv2.imread(imagePath)

mountingCluster = getMountingPoints(img)
mountingPointsList, mountingPointsImg = cluster2Point(mountingCluster, img)

cv2.imshow("mounting points as pixels", mountingPointsImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [171]:
# angle of camera from negative of vertical axisS
alpha = 60
ALPHA = alpha*math.pi/180

for mountingPoint in mountingPointsList:
    for beta in range (0,360, 10):

        # beta edge cases
        if(beta==180 or beta==0 or beta==360):
            continue

        # loop vars
        BETA = beta*math.pi/180
        roadCopy = road.copy()
        imgCopy = img.copy()

        # distances of closer and further edges from mounting point
        closer_dist = heightPix[20]*math.tan(ALPHA - (phi/2))
        further_dist = heightPix[20]*math.tan(ALPHA + (phi/2))

        # slope of horizontal plane camera angle
        slope_beta = math.tan(BETA)

        # midpoints of closer and further edges
        if beta > 180:
            closer_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, closer_dist)[1]
            further_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, further_dist)[1]
        else:
            closer_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, closer_dist)[0]
            further_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, further_dist)[0]

        # half of the closer edge length = (heightPix[20]*tan(w/2))/cos(alpha-(phi/2))
        closer_edge = (heightPix[20]*math.tan(omega/2))/math.cos(ALPHA-(phi/2))
        # half of the further edge length = (heightPix[20]*tan(w/2))/cos(alpha+(phi/2))
        further_edge =(heightPix[20]*math.tan(omega/2))/math.cos(ALPHA+(phi/2))

        # Obtaining on ground quadilateral points
        point1 = pointGen(Point(closer_midPoint[0],closer_midPoint[1]), -1/slope_beta, closer_edge)[0]
        point2 = pointGen(Point(closer_midPoint[0],closer_midPoint[1]), -1/slope_beta, closer_edge)[1]
        point3 = pointGen(Point(further_midPoint[0],further_midPoint[1]), -1/slope_beta, further_edge)[1]
        point4 = pointGen(Point(further_midPoint[0],further_midPoint[1]), -1/slope_beta, further_edge)[0]

        # plotting the points
        pt = np.array([point1, point2, point3, point4], np.int32)
        pt = pt.reshape((-1,1,2))
        cv2.fillPoly(imgCopy, [pt], (251,251,251))

        # mask the polygon
        polyMask = cv2.inRange(imgCopy, (251,251,251), (251,251,251))
        subtractedMask = np.subtract(roadCopy, polyMask)

        # find the updated are of camera coverage
        subtractedContour = cv2.findContours(subtractedMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        subtractedContour = subtractedContour[0] if len(subtractedContour) == 2 else subtractedContour[1]

        area_sum = 0
        for contour in subtractedContour:
            area = cv2.contourArea(contour)
            area_sum += area

        if area_sum > minArea:
            minArea = area_sum
            fp1 = point1
            fp2 = point2
            fp3 = point3
            fp4 = point4
            mp = mountingPoint

# plotting best shape and the mounting point
pts = np.array([fp1, fp2, fp3, fp4], np.int32)
pts = pts.reshape((-1,1,2))
cv2.polylines(img, [pts], True, (255,255,0))
cv2.circle(img, (mp[0], mp[1]), 5, (255,0,0),2)

# showing image
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()